In [1]:
import os

# 設定 nnU-Net 資料夾路徑
base_dir = "practice_nnunet"
nnUNet_raw = os.path.join(base_dir, "nnUNet_raw")
nnUNet_preprocessed = os.path.join(base_dir, "nnUNet_preprocessed")
nnUNet_results = os.path.join(base_dir, "nnUNet_results")

# 建立資料夾
os.makedirs(nnUNet_raw, exist_ok=True)
os.makedirs(nnUNet_preprocessed, exist_ok=True)
os.makedirs(nnUNet_results, exist_ok=True)

# 設定環境變數
os.environ["nnUNet_raw"] = nnUNet_raw
os.environ["nnUNet_preprocessed"] = nnUNet_preprocessed
os.environ["nnUNet_results"] = nnUNet_results

print("nnU-Net 資料夾已建立在 practice_nnunet/ 下")


nnU-Net 資料夾已建立在 practice_nnunet/ 下


In [ ]:
import os
import shutil
import glob

# -------------------------------
# nnU-Net 資料夾
# -------------------------------
task_dir = "/home/sandy0317/practice_nnUNet/practice_nnunet/nnUNet_raw/Dataset001" #改成自己的帳號名稱放置路徑中
imagesTr_dir = os.path.join(task_dir, "imagesTr")
labelsTr_dir = os.path.join(task_dir, "labelsTr")
imagesTs_dir = os.path.join(task_dir, "imagesTs")
os.makedirs(imagesTr_dir, exist_ok=True)
os.makedirs(labelsTr_dir, exist_ok=True)
os.makedirs(imagesTs_dir, exist_ok=True)

# -------------------------------
# 原始影像與 mask 資料夾
# -------------------------------
src_images = "/home/sandy0317/Public/train/med-ddpm-1/image" #改成自己的帳號名稱放置路徑中
src_masks = "/home/sandy0317/Public/train/med-ddpm-1/mask" #改成自己的帳號名稱放置路徑中

# -------------------------------
# 找出所有影像檔
# -------------------------------
image_paths = sorted(glob.glob(os.path.join(src_images, "*")))
mask_paths = sorted(glob.glob(os.path.join(src_masks, "*")))

# -------------------------------
# 檢查數量一致
# -------------------------------
assert len(image_paths) == len(mask_paths), f"影像數量({len(image_paths)})與標註數量({len(mask_paths)})不一致！"

# -------------------------------
# 複製並重新命名
# -------------------------------
for idx, (img_path, mask_path) in enumerate(zip(image_paths, mask_paths)):
    case_id = f"seg_{idx:04d}"

    # nnU-Net 格式檔名
    new_img_name = f"{case_id}_0000.nii.gz"  # 影像加 _0000
    new_mask_name = f"{case_id}.nii.gz"      # mask 保留 case_id

    shutil.copy(img_path, os.path.join(imagesTr_dir, new_img_name))
    shutil.copy(mask_path, os.path.join(labelsTr_dir, new_mask_name))

print(f"✅ 已完成轉換並複製 {len(image_paths)} 筆資料到 nnU-Net 格式資料夾：{task_dir}")


In [2]:
# ===============================
# nnU-Net Dataset 前置流程 (Jupyter Notebook)
# ===============================
#執行前請先至Terminal確認自己的nnU-Net可執行檔的位置，如語法which nnUNetv2_plan_and_preprocess

import os
import json
import glob
import shutil
import nibabel as nib
import numpy as np
import subprocess

# -------------------------------
# 1️⃣ 原始資料集路徑
# -------------------------------
task_dir = "/home/sandy0317/practice_nnUNet/practice_nnunet/nnUNet_raw/Dataset001" #改成自己的帳號名稱放置路徑中
imagesTr_dir = os.path.join(task_dir, "imagesTr")
labelsTr_dir = os.path.join(task_dir, "labelsTr")

# -------------------------------
# 2️⃣ 自動修正標籤檔：將 2 映射成 1
# -------------------------------
label_files = sorted(glob.glob(os.path.join(labelsTr_dir, "*.nii.gz")))
for f in label_files:
    img = nib.load(f)
    data = img.get_fdata()
    data[data == 2] = 1
    nib.Nifti1Image(data.astype(np.uint8), img.affine, img.header).to_filename(f)
print("✅ 標籤檔已修正完成 (2→1)")

# -------------------------------
# 3️⃣ 生成 dataset.json
# -------------------------------
image_files = sorted(glob.glob(os.path.join(imagesTr_dir, "*.nii.gz")))

dataset_json = {
    "name": "Dataset001",
    "description": "Segmentation",
    "tensorImageSize": "3D",
    "modality": {"0": "CT"},
    "labels": {"background": 0, "seg": 1},  # ✅ 修正
    "numTraining": len(image_files),
    "numTest": 0,
    "file_ending": ".nii.gz",
    "channel_names": {"0": "CT"},
    "training": [
        {
            "image": os.path.join("imagesTr", os.path.basename(img)),
            "label": os.path.join("labelsTr", os.path.basename(lbl))
        }
        for img, lbl in zip(image_files, label_files)
    ],
    "test": []
}


dataset_json_path = os.path.join(task_dir, "dataset.json")
with open(dataset_json_path, "w", encoding="utf-8") as f:
    json.dump(dataset_json, f, indent=4, ensure_ascii=False)
print("✅ dataset.json 已生成，背景標籤正確")

# -------------------------------
# 4️⃣ 設定 nnU-Net 環境變數
# -------------------------------
nnunet_raw = "/home/sandy0317/practice_nnUNet/practice_nnunet/nnUNet_raw" #改成自己的帳號名稱放置路徑中
nnunet_preprocessed = "/home/sandy0317/practice_nnUNet/practice_nnunet/nnUNet_preprocessed" #改成自己的帳號名稱放置路徑中
nnunet_results = "/home/sandy0317/practice_nnunet/nnUNet_results" #改成自己的帳號名稱放置路徑中

os.environ["nnUNet_raw"] = nnunet_raw
os.environ["nnUNet_preprocessed"] = nnunet_preprocessed
os.environ["RESULTS_FOLDER"] = nnunet_results
os.environ["nnUNet_results"] = nnunet_results  # ✅ 必須

# 建立資料夾
for p in [nnunet_raw, nnunet_preprocessed, nnunet_results]:
    os.makedirs(p, exist_ok=True)
    print(f"✅ 資料夾確認: {p}")

# -------------------------------
# 5️⃣ 執行 nnU-Net 資料集預處理 (CLI)
# -------------------------------

cmd = [
    "/home/sandy0317/.local/bin/nnUNetv2_plan_and_preprocess", #改成自己的帳號名稱放置路徑中
    "-p", task_dir,            # 指定 Task 資料夾完整路徑
    "--verify_dataset_integrity"
]

print("🚀 開始 nnU-Net 資料集預處理...")

#改成自己的帳號名稱放置路徑中
#!/home/sandy0317/.conda/envs/nnunet/bin/nnUNetv2_plan_and_preprocess -d 1 --verify_dataset_integrity
!/home/sandy0317/.local/bin/nnUNetv2_plan_and_preprocess -d 1 --verify_dataset_integrity
print("✅ nnU-Net 資料集預處理完成")

✅ 標籤檔已修正完成 (2→1)
✅ dataset.json 已生成，背景標籤正確
✅ 資料夾確認: /home/sandy0317/practice_nnUNet/practice_nnunet/nnUNet_raw
✅ 資料夾確認: /home/sandy0317/practice_nnUNet/practice_nnunet/nnUNet_preprocessed
✅ 資料夾確認: /home/sandy0317/practice_nnunet/nnUNet_results
🚀 開始 nnU-Net 資料集預處理...
Fingerprint extraction...
Dataset001
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer
100%|███████████████████████████████████████| 1000/1000 [00:59<00:00, 16.87it/s]
Experiment planning...

############################
INFO: You are using the old nnU-Net default planner. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
##################

In [11]:
#注意開始訓練前請記得至Terminal確認自己是否有安裝 GCC/G++ 編譯器(C 編譯器)
#若無C編譯器，conda activate進入環境後執行
#conda install -y -c conda-forge graphviz
#conda install -y -c conda-forge gcc_linux-64 gxx_linux-64
#指定環境變數
#export CC=$CONDA_PREFIX/bin/x86_64-conda-linux-gnu-gcc
#export CXX=$CONDA_PREFIX/bin/x86_64-conda-linux-gnu-g++

import os

# 先設定 CUDA_VISIBLE_DEVICES
os.environ["CUDA_VISIBLE_DEVICES"] = "2"  # 這裡改你想用的 GPU 編號

# 再設定 nnU-Net 環境變數
os.environ['CC'] = "/home/sandy0317/.conda/envs/nnunet/bin/x86_64-conda-linux-gnu-gcc"
os.environ['CXX'] = "/home/sandy0317/.conda/envs/nnunet/bin/x86_64-conda-linux-gnu-g++"
os.environ['nnUNet_raw'] = '/home/sandy0317/practice_nnUNet/practice_nnunet/nnUNet_raw'
os.environ['nnUNet_preprocessed'] = '/home/sandy0317/practice_nnUNet/practice_nnunet/nnUNet_preprocessed'
os.environ['nnUNet_results'] = '/home/sandy0317/practice_nnUNet/practice_nnunet/nnUNet_results'

os.environ['C_INCLUDE_PATH'] = "/home/sandy0317/.conda/envs/nnunet/include/python3.10"
os.environ['CPLUS_INCLUDE_PATH'] = "/home/sandy0317/.conda/envs/nnunet/include/python3.10"
os.environ['LD_LIBRARY_PATH'] = "/home/sandy0317/.conda/envs/nnunet/lib:" + os.environ.get('LD_LIBRARY_PATH','')

# 這行要放在環境變數都設定好「之後」
!echo "CUDA_VISIBLE_DEVICES=$CUDA_VISIBLE_DEVICES"

#改成自己的帳號名稱放置路徑中，並更改指定GPU 編號
#!/home/sandy0317/.conda/envs/nnunet/bin/nnUNetv2_train Dataset001 3d_fullres 0 -tr nnUNetTrainer_10epochs
!CUDA_VISIBLE_DEVICES=2 /home/sandy0317/.local/bin/nnUNetv2_train Dataset001 3d_fullres 0 -tr nnUNetTrainer_10epochs

CUDA_VISIBLE_DEVICES=2

############################
INFO: You are using the old nnU-Net default plans. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2025-10-17 08:51:37.559062: Using torch.compile...
2025-10-17 08:51:40.368879: do_dummy_2d_data_aug: False
2025-10-17 08:51:40.375142: Using splits from existing split file: /home/sandy0317/practice_nnUNet/practice_nnunet/nnUNet_preprocessed/Dataset001/splits_final.json
20

In [14]:
# ===========================
# 6. 推論 (Inference)
# ===========================
import os

# 先設定 CUDA_VISIBLE_DEVICES
os.environ["CUDA_VISIBLE_DEVICES"] = "6"  # 這裡改你想用的 GPU 編號

#改成自己的帳號名稱放置路徑中
os.environ['nnUNet_raw'] = '/home/sandy0317/practice_nnUNet/practice_nnunet/nnUNet_raw'
os.environ['nnUNet_preprocessed'] = '/home/sandy0317/practice_nnUNet/practice_nnunet/nnUNet_preprocessed'
os.environ['nnUNet_results'] = '/home/sandy0317/practice_nnUNet/practice_nnunet/nnUNet_results'

# 從環境變數中讀取路徑
nnUNet_raw = os.environ['nnUNet_raw']

# 測試影像資料夾
input_folder = os.path.join(nnUNet_raw, "Dataset001", "imagesTs")

# 改成相對路徑或家目錄下
output_folder = 'nnUNet_predictions'  # 或者 os.path.expanduser('~/nnUNet_predictions')


os.makedirs(output_folder, exist_ok=True)

# 這行要放在環境變數都設定好「之後」
!echo "CUDA_VISIBLE_DEVICES=$CUDA_VISIBLE_DEVICES"

#改成自己的帳號名稱放置路徑中，並更改指定GPU 編號
#!/home/sandy0317/.conda/envs/nnunet/bin/nnUNetv2_predict -d 1 -i $input_folder -o $output_folder -tr nnUNetTrainer_10epochs -c 3d_fullres -f 0
!CUDA_VISIBLE_DEVICES=6 /home/sandy0317/.local/bin/nnUNetv2_predict -d 1 -i $input_folder -o $output_folder -tr nnUNetTrainer_10epochs -c 3d_fullres -f 0

print("推論完成，結果存於：", output_folder)



CUDA_VISIBLE_DEVICES=6

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

There are 3 cases in the source folder
I am processing 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 3 cases that I would like to predict

Predicting sample_1:
perform_everything_on_device: True
100%|█████████████████████████████████████████████| 1/1 [00:02<00:00,  2.80s/it]
sending off prediction to background worker for resampling and export
done with sample_1

Predicting sample_2:
perform_everything_on_device: True
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  5.04it/s]
sending off prediction to background worker for

In [ ]:
# ===========================
# 7. 驗證 (可選)
# ===========================

# 先設定 CUDA_VISIBLE_DEVICES
os.environ["CUDA_VISIBLE_DEVICES"] = "6"  # 這裡改你想用的 GPU 編號

# 這行要放在環境變數都設定好「之後」
!echo "CUDA_VISIBLE_DEVICES=$CUDA_VISIBLE_DEVICES"

#改成自己的帳號名稱放置路徑中，並更改指定GPU 編號
#!/home/sandy0317/.conda/envs/nnunet/bin/nnUNetv2_evaluate_folder -ref $nnUNet_raw/Dataset001/labelsTr -pred $output_folder
!CUDA_VISIBLE_DEVICES=6 /home/sandy0317/.local/bin/nnUNetv2_evaluate_folder -ref $nnUNet_raw/Dataset001/labelsTr -pred $output_folder
